<a href="https://colab.research.google.com/github/universeatmyfeet/NLP/blob/master/Classwork/NLP_CW_J051_19thDec_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import itertools
import pandas as pd
import string
from gensim import models
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations 
from string import digits


In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")

In [6]:
df['TweetText'][1]

'@Apple will be adding more carrier support to the iPhone 4S (just announced)'

In [0]:
stop_words = set(stopwords.words('english'))

**Step 1: getting a set of unique words**

In [0]:
def toktoktok(text):
  text = text.translate(text.maketrans('','',string.punctuation))#removing punctuation
  text = text.translate(text.maketrans('','',digits))#removing digits
  t = word_tokenize(text)
  tokens = [word for word in t if word not in stop_words]#removing stop words
  #didnot lemmatize to get more precise similarity score
  return tokens

In [0]:
df['TweetTokens']= df['TweetText'].apply(toktoktok)


In [10]:
df['TweetTokens'][1]

['Apple', 'adding', 'carrier', 'support', 'iPhone', 'S', 'announced']

**Step 2 : Getting combination of words**

In [0]:
list_of_tokens = list(itertools.chain.from_iterable(df['TweetTokens']))# chain.from_iterable(['ABC', 'DEF']) --> A B C D E F

In [25]:
len(list_of_tokens)

57385

In [0]:
combos = list(combinations(list_of_tokens[30:45], 2))
#This function takes an iterable,here, list of tokens and a integer. 
#This will create all the unique combinations that have 2 members.

In [37]:
len(combos)

105

In [38]:
combos = pd.DataFrame(combos) # converted combinations to a dataframe
combos[0:15]

,0,1
0,switch,Apple
1,switch,iPhone
2,switch,See
3,switch,ya
4,switch,I
5,switch,realized
6,switch,reason
7,switch,I
8,switch,got
9,switch,twitter


In [0]:
#Trying permutations of words

In [0]:
from itertools import permutations

In [0]:
perms = list(permutations(list_of_tokens[30:45], 2))

In [33]:
len(perms) #different from length of combos; input had same no of words

210

In [34]:
perms[0:15]

[('switch', 'Apple'),
 ('switch', 'iPhone'),
 ('switch', 'See'),
 ('switch', 'ya'),
 ('switch', 'I'),
 ('switch', 'realized'),
 ('switch', 'reason'),
 ('switch', 'I'),
 ('switch', 'got'),
 ('switch', 'twitter'),
 ('switch', 'ios'),
 ('switch', 'thanks'),
 ('switch', 'apple'),
 ('switch', 'Im'),
 ('Apple', 'switch')]

**Step 3 : Exploring Word2Vec**

In [0]:
word_vec = Word2Vec(df['TweetTokens'], min_count=1)
words = list(word_vec.wv.vocab) # gives all unique words


In [53]:
words[:10]

['Now',
 'Apple',
 'get',
 'swype',
 'iphone',
 'crack',
 'Iphone',
 'adding',
 'carrier',
 'support']

In [0]:
simili = word_vec.wv.most_similar('apps')


In [63]:
simili

[('apple', 0.9944494962692261),
 ('que', 0.9941447377204895),
 ('Microsoft', 0.9940025210380554),
 ('iOS', 0.9939985871315002),
 ('Twitter', 0.9939713478088379),
 ('I', 0.9938588738441467),
 ('like', 0.9938538670539856),
 ('twitter', 0.993852972984314),
 ('RT', 0.9938497543334961),
 ('de', 0.993841290473938)]

**Cosine similarity using CBOW**

In [0]:
model1 = Word2Vec(df['TweetTokens'], min_count = 1, size = 100, window = 5) 

In [0]:
col = list(combos)


In [85]:
for j in range(combos.shape[0]):
    print(combos[0][j],combos[1][j],' : ',model1.wv.similarity(combos[0][j],combos[1][j]))


switch Apple  :  0.74719155
switch iPhone  :  0.7518188
switch See  :  0.67265636
switch ya  :  0.7452325
switch I  :  0.7499093
switch realized  :  0.25931194
switch reason  :  0.039743092
switch I  :  0.7499093
switch got  :  0.7392025
switch twitter  :  0.748876
switch ios  :  0.7389795
switch thanks  :  0.7095501
switch apple  :  0.7458699
switch Im  :  0.74784756
Apple iPhone  :  0.99864346
Apple See  :  0.9048264
Apple ya  :  0.99603176
Apple I  :  0.9993758
Apple realized  :  0.3956061
Apple reason  :  0.23217747
Apple I  :  0.9993758
Apple got  :  0.9965213
Apple twitter  :  0.9993134
Apple ios  :  0.99345964
Apple thanks  :  0.97505087
Apple apple  :  0.9994122
Apple Im  :  0.9988056
iPhone See  :  0.9023573
iPhone ya  :  0.99571717
iPhone I  :  0.9989594
iPhone realized  :  0.3938801
iPhone reason  :  0.23362765
iPhone I  :  0.9989594
iPhone got  :  0.9956518
iPhone twitter  :  0.9985929
iPhone ios  :  0.99260443
iPhone thanks  :  0.974578
iPhone apple  :  0.9987272
iPhone Im

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


**Cosine Similarity using Skip-Gram**

In [0]:
model2 = Word2Vec(df['TweetTokens'], min_count=1,size= 50,workers=3, window =3, sg = 1)

In [89]:
for j in range(combos.shape[0]):
    print(combos[0][j],combos[1][j],' : ',model2.wv.similarity(combos[0][j],combos[1][j]))


switch Apple  :  0.97039646
switch iPhone  :  0.9703878
switch See  :  0.9647688
switch ya  :  0.96907735
switch I  :  0.9642332
switch realized  :  0.9000419
switch reason  :  0.37768263
switch I  :  0.9642332
switch got  :  0.96601754
switch twitter  :  0.9658686
switch ios  :  0.96548265
switch thanks  :  0.9592208
switch apple  :  0.9631445
switch Im  :  0.96477026
Apple iPhone  :  0.9958934
Apple See  :  0.9934478
Apple ya  :  0.99629813
Apple I  :  0.99345756
Apple realized  :  0.94422215
Apple reason  :  0.45287564
Apple I  :  0.99345756
Apple got  :  0.99588716
Apple twitter  :  0.9926192
Apple ios  :  0.9956795
Apple thanks  :  0.9943278
Apple apple  :  0.9933578
Apple Im  :  0.99178207
iPhone See  :  0.9882439
iPhone ya  :  0.99088794
iPhone I  :  0.9924425
iPhone realized  :  0.9393633
iPhone reason  :  0.45168808
iPhone I  :  0.9924425
iPhone got  :  0.9921688
iPhone twitter  :  0.98806787
iPhone ios  :  0.991508
iPhone thanks  :  0.9914447
iPhone apple  :  0.99106133
iPhon

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
